In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv('jantojun2020.csv', dtype=object)
existing = pd.read_csv('airlines-corgis.csv')

In [16]:
df.head(5).iloc[:,0:]

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_UNIQUE_CARRIER,MKT_CARRIER_FL_NUM,TAIL_NUM,ORIGIN,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2020,1,1,1,3,1/1/2020,WN,5888,N951WN,ONT,...,95.0,122.0,74.0,363,2,8.0,0.0,27.0,0.0,33.0
1,2020,1,1,1,3,1/1/2020,WN,6276,N467WN,ONT,...,90.0,92.0,71.0,363,2,NaN,NaN,NaN,NaN,NaN
2,2020,1,1,1,3,1/1/2020,WN,4598,N7885A,ONT,...,70.0,68.0,57.0,333,2,NaN,NaN,NaN,NaN,NaN
3,2020,1,1,1,3,1/1/2020,WN,4761,N551WN,ONT,...,75.0,75.0,63.0,333,2,NaN,NaN,NaN,NaN,NaN
4,2020,1,1,1,3,1/1/2020,WN,5162,N968WN,ONT,...,80.0,67.0,57.0,333,2,NaN,NaN,NaN,NaN,NaN


In [17]:
existing.head(5)
existing.iloc[0:5, 7:]

#existing['Statistics.Carriers.Names'].str.split(",", expand=True)
#Tells me max number of airlines
#existing['Statistics.Carriers.Names'].str.split(",", expand=True).notna().cumsum(axis=1).max(axis=1).max()

,Statistics.# of Delays.Late Aircraft,Statistics.# of Delays.National Aviation System,Statistics.# of Delays.Security,Statistics.# of Delays.Weather,Statistics.Carriers.Names,Statistics.Carriers.Total,Statistics.Flights.Cancelled,Statistics.Flights.Delayed,Statistics.Flights.Diverted,Statistics.Flights.On Time,Statistics.Flights.Total,Statistics.Minutes Delayed.Carrier,Statistics.Minutes Delayed.Late Aircraft,Statistics.Minutes Delayed.National Aviation System,Statistics.Minutes Delayed.Security,Statistics.Minutes Delayed.Total,Statistics.Minutes Delayed.Weather
0,1275,3217,17,328,"American Airlines Inc.,JetBlue Airways,Contine...",11,216,5843,27,23974,30060,61606,68335,118831,518,268764,19474
1,495,685,3,66,"American Airlines Inc.,Alaska Airlines Inc.,Co...",14,138,1623,3,7875,9639,20319,28189,24400,99,77167,4160
2,477,389,8,78,"American Airlines Inc.,Continental Air Lines I...",11,29,1245,15,6998,8287,13635,26810,17556,278,64480,6201
3,472,735,2,54,"American Airlines Inc.,Continental Air Lines I...",11,73,1562,14,7021,8670,14763,23379,23804,127,65865,3792
4,268,487,4,58,"American Airlines Inc.,Alaska Airlines Inc.,Co...",13,74,1100,18,5321,6513,13775,13712,20999,120,52747,4141


In [18]:
#Start similar dataframe from new data
new = pd.DataFrame()
new['Airport.Code'] = df["ORIGIN"]

#Create dictionary of airport codes : airport names
codes = pd.Series(existing['Airport.Name'].values, existing['Airport.Code'].values).to_dict()

#Add airport names to the new dataframe
L = list(codes.keys()) + list(codes.values())
new['Airport.Name'] = new['Airport.Code'].str.extract('(' + '|'.join(L) + ')', expand=False).replace(codes)

In [19]:
#Add date information
new['Time.Label'] = df['YEAR'] + '/0' + df['MONTH']
new['Time.Month'] = df['MONTH']
months = {'1':'January', '2':'February', '3':'March', '4':'April', '5':'May', '6':'June'}
new['Time.Month Name'] = new['Time.Month'].replace(months)
new['Time.Year'] = df['YEAR']

#Create dataframe that will hold info about each individual flight
new_indiv = new.copy(deep=True)

#Ignore flights from airports not in original dataset
new = new.dropna(subset=['Airport.Name'])

#Reduce rows to one for each Airport.Code/Time.Label combo (like exisiting dataframe)
new = new.drop_duplicates(subset=['Airport.Code', 'Time.Label'])

In [20]:
#Copy over data from jantojun2020... Currently in delay per minutes
new_indiv['Statistics.# of Delays.Carrier'] = df['CARRIER_DELAY']
new_indiv['Statistics.# of Delays.Late Aircraft'] = df['LATE_AIRCRAFT_DELAY']
new_indiv['Statistics.# of Delays.National Aviation System'] = df['NAS_DELAY']
new_indiv['Statistics.# of Delays.Security'] = df['SECURITY_DELAY']
new_indiv['Statistics.# of Delays.Weather'] = df['WEATHER_DELAY']

#Replace all delays of 0.0 minutes (no delays) with 0, otherwise replace with 1 (meaning there was a delay)
new_indiv['Statistics.# of Delays.Carrier'] = [1 if float(x) > 0.0 else 0 for x in new_indiv['Statistics.# of Delays.Carrier']]
new_indiv['Statistics.# of Delays.Late Aircraft'] = [1 if float(x) > 0.0 else 0 for x in new_indiv['Statistics.# of Delays.Late Aircraft']]
new_indiv['Statistics.# of Delays.National Aviation System'] = [1 if float(x) > 0.0 else 0 for x in new_indiv['Statistics.# of Delays.National Aviation System']]
new_indiv['Statistics.# of Delays.Security'] = [1 if float(x) > 0.0 else 0 for x in new_indiv['Statistics.# of Delays.Security']]
new_indiv['Statistics.# of Delays.Weather'] = [1 if float(x) > 0.0 else 0 for x in new_indiv['Statistics.# of Delays.Weather']]

In [21]:
#Create dict to map airline codes to names
airlines = {'AA':'American Airlines Inc.', 'AS':'Alaska Airlines Inc.', 'B6':'JetBlue Airways', 
            'DL':'Delta Air Lines Inc.', 'F9':'Frontier Airlines Inc.', 'G4': np.nan, 'HA':'Hawaiian Airlines Inc.',
            'NK':'Spirit Air Lines', 'UA':'United Air Lines Inc.', 'WN': 'Southwest Airlines Co.'}

#Transfer over airline names
new_indiv['Statistics.Carriers.Names'] = df['MKT_UNIQUE_CARRIER']

#Replace airline codes with names. Allegiant airlines not in original dataset, so its name is repalced with NaN
new_indiv['Statistics.Carriers.Names'] = new_indiv['Statistics.Carriers.Names'].replace(airlines)
pd.set_option('display.max_rows', 10)

In [24]:
#Transfer cancelled flights data
new_indiv['Statistics.Flights.Cancelled'] = df['CANCELLED'].astype('int64')

#Add delayed flights data (1=Delayed, 0=Not Delayed)
df['DEP_DELAY_NEW'] = df['DEP_DELAY_NEW'].astype('float64')
new_indiv['Statistics.Flights.Delayed'] = [1 if x > 0 else 0 for x in df['DEP_DELAY_NEW']]
#[x if y == 0 else 0 for x in new_indiv['Statistics.Flights.Delayed'] for y in new_indiv['Statistics.Flights.Cancelled']]

#No data in the new dataset on diverted or on-time flights
#We cannot account for diverted, but we will count on-time as being neither cancelled or delayed
#(1 = On-Time, 0 = Not On-Time)
new_indiv['Statistics.Flights.On Time'] = new_indiv['Statistics.Flights.Cancelled'] + new_indiv['Statistics.Flights.Delayed']
new_indiv['Statistics.Flights.On Time'] = new_indiv['Statistics.Flights.On Time'].replace(2, 1)
new_indiv['Statistics.Flights.On Time'] = [1-x for x in new_indiv['Statistics.Flights.On Time']]

#Every flight that is cancelled has its delay status set to 0 (for sake of not double-counting in totals)
new_indiv.loc[new_indiv['Statistics.Flights.Cancelled'] > 0, 'Statistics.Flights.Delayed'] = 0

In [26]:
#Transfer data of minutes of delays over
new_indiv['Statistics.Minutes Delayed.Carrier'] = df['CARRIER_DELAY']
new_indiv['Statistics.Minutes Delayed.Late Aircraft'] = df['LATE_AIRCRAFT_DELAY']
new_indiv['Statistics.Minutes Delayed.National Aviation System'] = df['NAS_DELAY']
new_indiv['Statistics.Minutes Delayed.Security'] = df['SECURITY_DELAY']
new_indiv['Statistics.Minutes Delayed .Weather'] = df['WEATHER_DELAY']
new_indiv

,Airport.Code,Airport.Name,Time.Label,Time.Month,Time.Month Name,Time.Year,Statistics.# of Delays.Carrier,Statistics.# of Delays.Late Aircraft,Statistics.# of Delays.National Aviation System,Statistics.# of Delays.Security,Statistics.# of Delays.Weather,Statistics.Carriers.Names,Statistics.Flights.Cancelled,Statistics.Flights.Delayed,Statistics.Flights.On Time,Statistics.Minutes Delayed.Carrier,Statistics.Minutes Delayed.Late Aircraft,Statistics.Minutes Delayed.National Aviation System,Statistics.Minutes Delayed.Security,Statistics.Minutes Delayed .Weather
0,ONT,NaN,2020/01,1,January,2020,1,1,1,0,0,Southwest Airlines Co.,0,1,0,8.0,33.0,27.0,0.0,0.0
1,ONT,NaN,2020/01,1,January,2020,0,0,0,0,0,Southwest Airlines Co.,0,0,1,NaN,NaN,NaN,NaN,NaN
2,ONT,NaN,2020/01,1,January,2020,0,0,0,0,0,Southwest Airlines Co.,0,0,1,NaN,NaN,NaN,NaN,NaN
3,ONT,NaN,2020/01,1,January,2020,0,0,0,0,0,Southwest Airlines Co.,0,1,0,NaN,NaN,NaN,NaN,NaN
4,ONT,NaN,2020/01,1,January,2020,0,0,0,0,0,Southwest Airlines Co.,0,1,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745842,SEA,"Seattle, WA: Seattle/Tacoma International",2020/06,6,June,2020,0,0,0,0,0,Alaska Airlines Inc.,0,0,1,NaN,NaN,NaN,NaN,NaN
2745843,PHX,"Phoenix, AZ: Phoenix Sky Harbor International",2020/06,6,June,2020,0,0,0,0,0,Alaska Airlines Inc.,0,0,1,NaN,NaN,NaN,NaN,NaN
2745844,SEA,"Seattle, WA: Seattle/Tacoma International",2020/06,6,June,2020,0,0,0,0,0,Alaska Airlines Inc.,0,0,1,NaN,NaN,NaN,NaN,NaN
2745845,SNA,NaN,2020/06,6,June,2020,0,0,0,0,0,Alaska Airlines Inc.,0,0,1,NaN,NaN,NaN,NaN,NaN


In [27]:
pd.set_option('display.max_rows', None)
#Drop and ignore and flights that are at airports not in the original dataset
new_indiv = new_indiv.dropna(subset=['Airport.Name'])

#Sort "new" df entries by data, then by airport
new = new.sort_values(['Time.Label', 'Airport.Code'])

#Adding total carrier delay info for each airport/data combo entry
delays_car = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Statistics.# of Delays.Carrier'].sum())
new['Statistics.# of Delays.Carrier'] = delays_car

#Adding total late aircraft delay info for each airport/data combo entry
delays_late = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Statistics.# of Delays.Late Aircraft'].sum())
new['Statistics.# of Delays.Late Aircraft'] = delays_late

#Adding total national aviation system delay info for each airport/data combo entry
delays_nas = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Statistics.# of Delays.National Aviation System'].sum())
new['Statistics.# of Delays.National Aviation System'] = delays_nas

#Adding total security delay info for each airport/data combo entry
delays_sec = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Statistics.# of Delays.Security'].sum())
new['Statistics.# of Delays.Security'] = delays_sec

#Adding total weather delay info for each airport/data combo entry
delays_wthr = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Statistics.# of Delays.Weather'].sum())
new['Statistics.# of Delays.Weather'] = delays_wthr

In [48]:
pd.set_option('display.max_rows', None)
#Added as a placeholder for now
new['Statistics.Carriers.Names'] = ''

#Add number of carriers for each airport/date combo 
carriers = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Statistics.Carriers.Names'].nunique())
new['Statistics.Carriers.Total'] = carriers

##Add flights delayed for each airport/date combo 
cancelled = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Statistics.Flights.Cancelled'].sum())
new['Statistics.Flights.Cancelled'] = cancelled

#Add flights cancelled for each airport/date combo
delays = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Statistics.Flights.Delayed'].sum())
new['Statistics.Flights.Delayed'] = delays

#Diverted flights arent in new dataset, so set to null
new['Statistics.Flights.Diverted'] = np.nan

#Placeholder for now
new['Statistics.Flights.On Time'] = ''

#Add total number of flights for each airport/date combo
totals = list(new_indiv.groupby(['Time.Label', 'Airport.Code'])['Airport.Name'].count())
new['Statistics.Flights.Total'] = totals

In [12]:
test = new.dropna(subset=['Airport.Name'])
test.reset_index(inplace=True)
test = test.drop('index', axis=1)
test['Time.Month'], test['Time.Year'] = [df['MONTH'], df['YEAR']]
test

,Airport.Code,Airport.Name,Time.Label,Time.Month,Time.Month Name,Time.Year,Statistics.# of Delays.Carrier,Statistics.# of Delays.Late Aircraft,Statistics.# of Delays.National Aviation System,Statistics.# of Delays.Security,Statistics.# of Delays.Weather,Statistics.Carriers.Names,Statistics.Carriers.Total
0,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2020/01,1,January,2020,1860,1039,1443,6,377,,8
1,BOS,"Boston, MA: Logan International",2020/01,1,January,2020,746,409,796,1,37,,9
2,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",2020/01,1,January,2020,592,391,474,4,19,,8
3,CLT,"Charlotte, NC: Charlotte Douglas International",2020/01,1,January,2020,2119,2046,2383,31,189,,7
4,DCA,"Washington, DC: Ronald Reagan Washington National",2020/01,1,January,2020,756,982,948,4,40,,7
5,DEN,"Denver, CO: Denver International",2020/01,1,January,2020,1574,1171,1220,8,129,,8
6,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",2020/01,1,January,2020,2592,2273,2038,10,525,,7
7,DTW,"Detroit, MI: Detroit Metro Wayne County",2020/01,1,January,2020,869,327,910,1,160,,8
8,EWR,"Newark, NJ: Newark Liberty International",2020/01,1,January,2020,1413,1324,1419,11,59,,7
9,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",2020/01,1,January,2020,737,564,715,15,23,,8
